In [1]:
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import json
charFile = 'characteristics.csv'
char = dd.read_csv(charFile, blocksize=700e6)

In [2]:
revCharFile = 'characteristic_reviews.csv'
revChar = dd.read_csv(revCharFile, blocksize=700e6)

In [3]:
revChar.head(10)

,id,characteristic_id,review_id,value
0,1,1,1,4
1,2,2,1,3
2,3,3,1,5
3,4,4,1,4
4,5,1,2,4
5,6,2,2,4
6,7,3,2,5
7,8,4,2,4
8,9,5,3,4
9,10,5,4,5


In [4]:
result = dd.merge(char, revChar, left_on='id', right_on='characteristic_id', how='inner')
result

,id_x,product_id,name,id_y,characteristic_id,review_id,value
npartitions=1,,,,,,,
,int64,int64,string,int64,int64,int64,int64
,...,...,...,...,...,...,...


In [5]:
char.head(10)

,id,product_id,name
0,1,1,Fit
1,2,1,Length
2,3,1,Comfort
3,4,1,Quality
4,5,2,Quality
5,6,3,Fit
6,7,3,Length
7,8,3,Comfort
8,9,3,Quality
9,10,4,Fit


In [6]:
test = result.drop(columns=['id_y'])

In [8]:
type(test)

dask.dataframe.core.DataFrame

In [10]:
test['result'] = test.map_partitions(lambda x: [{'name': a, 'characteristic_id': b, 'value': c} for (a, b, c) in zip(x['name'], x['characteristic_id'], x['value'])], meta=('result', 'object'))

In [11]:
test = test.drop(columns=['characteristic_id', 'id_x', 'name', 'value'])

In [12]:
test
test.head(20)

,product_id,review_id,result
0,1,1,"{'name': 'Fit', 'characteristic_id': 1, 'value..."
1,1,2,"{'name': 'Fit', 'characteristic_id': 1, 'value..."
2,1,1,"{'name': 'Length', 'characteristic_id': 2, 'va..."
3,1,2,"{'name': 'Length', 'characteristic_id': 2, 'va..."
4,1,1,"{'name': 'Comfort', 'characteristic_id': 3, 'v..."
5,1,2,"{'name': 'Comfort', 'characteristic_id': 3, 'v..."
6,1,1,"{'name': 'Quality', 'characteristic_id': 4, 'v..."
7,1,2,"{'name': 'Quality', 'characteristic_id': 4, 'v..."
8,2,3,"{'name': 'Quality', 'characteristic_id': 5, 'v..."
9,2,4,"{'name': 'Quality', 'characteristic_id': 5, 'v..."


In [14]:
def format(x):
    return json.dumps(list(x))

In [16]:
test = test.groupby(["review_id"], group_keys=True)["result"].apply(format, meta=('result', 'object'))

In [17]:
test

Dask Series Structure:
npartitions=1
    string
       ...
Name: result, dtype: string
Dask Name: to_pyarrow_string, 17 graph layers

In [19]:
sorted = test.to_frame()


In [21]:
sorted.compute()
sorted.to_csv("AM I DONE3.csv")

['/Users/Chris/hackreactor/Python/AM I DONE3.csv/0.part']